In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import pudl, ferc1, eia923, settings, constants
from pudl import models, models_ferc1, models_eia923
from pudl import clean_eia923, clean_ferc1, clean_pudl
from pudl import analysis
import numpy as np
import pandas as pd
import sqlalchemy as sa
import matplotlib.pyplot as plt
import matplotlib as mpl
pudl_engine = pudl.db_connect_pudl()
f1_engine = ferc1.db_connect_ferc1()
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['figure.dpi'] = 150

In [2]:
# Make sure we get an exhaustive list.
ferc1.init_db(refyear=2016, years=range(2004,2017), def_db=True)

Defining new FERC Form 1 DB based on 2016...
Clearing any existing FERC Form 1 database MetaData...
Ingesting FERC Form 1 Data from 2004...
Ingesting FERC Form 1 Data from 2005...
Ingesting FERC Form 1 Data from 2006...
Ingesting FERC Form 1 Data from 2007...
Ingesting FERC Form 1 Data from 2008...
Ingesting FERC Form 1 Data from 2009...
Ingesting FERC Form 1 Data from 2010...
Ingesting FERC Form 1 Data from 2011...
Ingesting FERC Form 1 Data from 2012...
Ingesting FERC Form 1 Data from 2013...
Ingesting FERC Form 1 Data from 2014...
Ingesting FERC Form 1 Data from 2015...
Ingesting FERC Form 1 Data from 2016...


In [3]:
new_fercplants = analysis.fercplants(new=True, years=[2016,], min_capacity=0.0)
pudl_fercutils = pd.read_sql('''select respondent_id, respondent_name from utilities_ferc;''', pudl_engine)
new_fercutils = new_fercplants[['respondent_id','respondent_name']].drop_duplicates()
new_fercutils = new_fercutils.set_index(['respondent_id',])
pudl_fercutils = pudl_fercutils.set_index(['respondent_id',])
new_index = new_fercutils.index.difference(pudl_fercutils.index)
really_new_fercutils = new_fercutils.loc[new_index].reset_index()
really_new_fercutils

Defining new FERC Form 1 DB based on 2015...
Clearing any existing FERC Form 1 database MetaData...


,respondent_id,respondent_name
0,513,"4C Acquisition, Llc"


In [4]:
small_plants_2016 = pd.read_sql('''SELECT report_year, respondent_id, row_number, spplmnt_num, plant_name, kind_of_fuel \
                                        FROM f1_gnrt_plant \
                                        WHERE report_year=2016;''', f1_engine)
small_plants_2016['record_number'] = small_plants_2016['row_number'] + 46*small_plants_2016['spplmnt_num']
small_plants_2016 = small_plants_2016.drop(['row_number','spplmnt_num'],axis=1)
small_plants_2016 = small_plants_2016[['report_year','respondent_id','record_number','plant_name','kind_of_fuel']]
small_plants_2016.to_excel('small_plants_2016.xlsx', index=False)

In [5]:
len(new_fercplants)

130

In [6]:
new_fercplants.to_excel('ferc1_unmapped_plants_2016.xlsx')

In [7]:
new_fercplants

,respondent_id,plant_name,respondent_name,plant_table
0,2,J H Miller - Total,Alabama Power Company,f1_steam
1,2,Lowndes County,Alabama Power Company,f1_steam
2,3,Industrial Blvd,Alaska Electric Light And Power Company,f1_steam
3,7,Foothills I & Ii,Arizona Public Service Company,f1_gnrt_plant
4,7,Gila Bend I,Arizona Public Service Company,f1_gnrt_plant
5,7,Gila Bend Ii,Arizona Public Service Company,f1_gnrt_plant
6,7,Hyder I,Arizona Public Service Company,f1_gnrt_plant
7,8,Union Power Station,"Entergy Arkansas, Inc.",f1_steam
8,11,Bar Harbor (Young'S Corner),Bangor Hydro-Electric Company,f1_gnrt_plant
9,12,Ben French Ct,"Black Hills Power, Inc.",f1_steam
